In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 1. sql 연결 설정
# .env 파일 로드
load_dotenv()

# 환경변수에서 읽기
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

# 연결 엔진 생성
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')
# print(engine)

# 2. 시각화 환경설정
# 한글 폰트 설정 (Windows)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 3. 데이터 로드
# 기존 RFM 세그먼트 정보
df_segments = pd.read_sql("SELECT * FROM rfm_customer_segments_v2", engine)
# 기존 RFM 요약 정보
df_summary = pd.read_sql("SELECT * FROM rfm_summary_v2", engine)
# 세그먼트별 상품 구매 내역(메인)
df_purchases = pd.read_sql("SELECT * FROM segment_product_purchases", engine)
# 세그먼트별 상품 요약
df_product_summary = pd.read_sql("SELECT * FROM segment_product_summary", engine)
# vip고객과 신규 고객 비교
df_champions_vs_new = pd.read_sql("SELECT * FROM champions_vs_new_products", engine)
# 세그먼트별 상위 상품
df_top_products = pd.read_sql("SELECT * FROM segment_top_products", engine)
# 세그먼트별 구매 패턴 요약
df_purchase_patterns = pd.read_sql("SELECT * FROM segment_purchase_patterns", engine)

print(f"전체 구매 내역: {len(df_purchases):,}건")
print(f"고객 수: {df_purchases['customerid'].nunique():,}명")
print(f"세그먼트 수: {df_purchases['segment_name'].nunique()}개")

# 4. 데이터 전처리

# 상품명 정제 (간단하게)
df_purchases['description_clean'] = df_purchases['description'].str.strip().str.upper()

# 세그먼트 순서 정의 (중요도 순)
segment_order = ['Champions', 'Loyal', 'Promising', 'New', 'Big', 
                 'Need Attention', 'Risk', 'Others', 'Hibernating', 'Lost']


In [ ]:
# 1. 세그먼트별 구매 패턴 분석 요약
print(df_purchase_patterns.to_string(index=False))

# 2. 세그먼트별 TOP 상품 분석(예시 Champions,New)
def get_top_products_by_segment(segment_name, top_n=10):
    """특정 세그먼트의 상위 N개 상품 반환"""
    segment_data = df_top_products[df_top_products['segment_name'] == segment_name]
    top_items = segment_data.nsmallest(top_n, 'revenue_rank')
    return top_items[['stockcode', 'description', 'total_revenue', 
                      'total_quantity', 'customer_count', 'revenue_rank']]

print("\n" + "="*80)
print("Champions 세그먼트 TOP 10 상품")
print("="*80)
print(get_top_products_by_segment('Champions', 10).to_string(index=False))

print("\n" + "="*80)
print("New 세그먼트 TOP 10 상품")
print("="*80)
print(get_top_products_by_segment('New', 10).to_string(index=False))


# 3. Champions vs New 비교 분석  (vip고객과 신규고객 차이 비교)
# 그룹별 집계
champions_new_summary = df_champions_vs_new.groupby('customer_group').agg({
    'total_revenue': 'sum',
    'customer_count': 'sum',
    'order_count': 'sum',
    'total_quantity': 'sum',
    'purchase_count': 'sum'
}).reset_index()

champions_new_summary['avg_revenue_per_customer'] = (
    champions_new_summary['total_revenue'] / champions_new_summary['customer_count']
)
champions_new_summary['avg_revenue_per_order'] = (
    champions_new_summary['total_revenue'] / champions_new_summary['order_count']
)

print("\n" + "="*80)
print("Champions vs New 전체 비교")
print("="*80)
print(champions_new_summary.to_string(index=False))

# Champions와 New의 TOP 상품 비교
champions_top = df_champions_vs_new[df_champions_vs_new['customer_group'] == 'Champions'].nlargest(20, 'total_revenue')
new_top = df_champions_vs_new[df_champions_vs_new['customer_group'] == 'New'].nlargest(20, 'total_revenue')

print("\n" + "="*80)
print("Champions만 선호하는 상품 (New는 구매 안함)")
print("="*80)
champions_only = champions_top[~champions_top['stockcode'].isin(new_top['stockcode'])]
print(champions_only[['stockcode', 'description', 'total_revenue', 'customer_count']].head(10).to_string(index=False))

print("\n" + "="*80)
print("New만 선호하는 상품 (Champions는 구매 안함)")
print("="*80)
new_only = new_top[~new_top['stockcode'].isin(champions_top['stockcode'])]
print(new_only[['stockcode', 'description', 'total_revenue', 'customer_count']].head(10).to_string(index=False))

In [ ]:
# 시각화

# 1. 세그먼트별 구매 패턴 비교 (전반적인 구매 특성 파악)
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 총 매출
ax1 = axes[0, 0]
data_sorted = df_purchase_patterns.sort_values('total_revenue', ascending=False)
sns.barplot(data=data_sorted, x='segment_name', y='total_revenue', 
            palette='viridis', ax=ax1)
ax1.set_title('세그먼트별 총 매출', fontsize=14, fontweight='bold')
ax1.set_xlabel('세그먼트', fontsize=12)
ax1.set_ylabel('총 매출 (£)', fontsize=12)
ax1.tick_params(axis='x', rotation=45)
for container in ax1.containers:
    ax1.bar_label(container, fmt='£%.0f', fontsize=9)

# 평균 구매액
ax2 = axes[0, 1]
data_sorted = df_purchase_patterns.sort_values('avg_purchase_value', ascending=False)
sns.barplot(data=data_sorted, x='segment_name', y='avg_purchase_value', 
            palette='coolwarm', ax=ax2)
ax2.set_title('세그먼트별 평균 구매액', fontsize=14, fontweight='bold')
ax2.set_xlabel('세그먼트', fontsize=12)
ax2.set_ylabel('평균 구매액 (£)', fontsize=12)
ax2.tick_params(axis='x', rotation=45)
for container in ax2.containers:
    ax2.bar_label(container, fmt='£%.1f', fontsize=9)

# 고객당 상품 다양성
ax3 = axes[1, 0]
data_sorted = df_purchase_patterns.sort_values('products_per_customer', ascending=False)
sns.barplot(data=data_sorted, x='segment_name', y='products_per_customer', 
            palette='rocket', ax=ax3)
ax3.set_title('세그먼트별 고객당 구매 상품 수', fontsize=14, fontweight='bold')
ax3.set_xlabel('세그먼트', fontsize=12)
ax3.set_ylabel('고객당 상품 수', fontsize=12)
ax3.tick_params(axis='x', rotation=45)
for container in ax3.containers:
    ax3.bar_label(container, fmt='%.1f', fontsize=9)

# 고객 수
ax4 = axes[1, 1]
data_sorted = df_purchase_patterns.sort_values('customer_count', ascending=False)
sns.barplot(data=data_sorted, x='segment_name', y='customer_count', 
            palette='mako', ax=ax4)
ax4.set_title('세그먼트별 고객 수', fontsize=14, fontweight='bold')
ax4.set_xlabel('세그먼트', fontsize=12)
ax4.set_ylabel('고객 수', fontsize=12)
ax4.tick_params(axis='x', rotation=45)
for container in ax4.containers:
    ax4.bar_label(container, fmt='%d', fontsize=9)

plt.tight_layout()
# plt.savefig('segment_purchase_patterns.png', dpi=300, bbox_inches='tight')
plt.show()



# 2. Champions vs New 비교 (최상위 고객과 신규 고객의 가치 차이 확인)
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# 총 매출 비교
ax1 = axes[0]
sns.barplot(data=champions_new_summary, x='customer_group', y='total_revenue', 
            palette=['#e74c3c', '#3498db'], ax=ax1)
ax1.set_title('총 매출 비교', fontsize=14, fontweight='bold')
ax1.set_xlabel('고객 그룹', fontsize=12)
ax1.set_ylabel('총 매출 (£)', fontsize=12)
for container in ax1.containers:
    ax1.bar_label(container, fmt='£%.0f', fontsize=11)

# 고객당 평균 매출
ax2 = axes[1]
sns.barplot(data=champions_new_summary, x='customer_group', y='avg_revenue_per_customer', 
            palette=['#e74c3c', '#3498db'], ax=ax2)
ax2.set_title('고객당 평균 매출', fontsize=14, fontweight='bold')
ax2.set_xlabel('고객 그룹', fontsize=12)
ax2.set_ylabel('고객당 평균 매출 (£)', fontsize=12)
for container in ax2.containers:
    ax2.bar_label(container, fmt='£%.1f', fontsize=11)

# 주문당 평균 매출
ax3 = axes[2]
sns.barplot(data=champions_new_summary, x='customer_group', y='avg_revenue_per_order', 
            palette=['#e74c3c', '#3498db'], ax=ax3)
ax3.set_title('주문당 평균 매출', fontsize=14, fontweight='bold')
ax3.set_xlabel('고객 그룹', fontsize=12)
ax3.set_ylabel('주문당 평균 매출 (£)', fontsize=12)
for container in ax3.containers:
    ax3.bar_label(container, fmt='£%.2f', fontsize=11)

plt.tight_layout()
# plt.savefig('champions_vs_new_comparison.png', dpi=300, bbox_inches='tight')
plt.show()




# 3. Champions vs New TOP 10 상품 비교( 각 그룹이 선호하는 상품 파악-> 추후 맞춤 추천 전략)
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Champions TOP 10
ax1 = axes[0]
champions_top10 = df_champions_vs_new[
    df_champions_vs_new['customer_group'] == 'Champions'
].nlargest(10, 'total_revenue')
champions_top10 = champions_top10.sort_values('total_revenue')
ax1.barh(range(len(champions_top10)), champions_top10['total_revenue'], color='#e74c3c')
ax1.set_yticks(range(len(champions_top10)))
ax1.set_yticklabels([f"{row['stockcode']}\n{row['description'][:30]}" 
                      for _, row in champions_top10.iterrows()], fontsize=9)
ax1.set_xlabel('총 매출 (£)', fontsize=12)
ax1.set_title('Champions TOP 10 상품', fontsize=14, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# New TOP 10
ax2 = axes[1]
new_top10 = df_champions_vs_new[
    df_champions_vs_new['customer_group'] == 'New'
].nlargest(10, 'total_revenue')
new_top10 = new_top10.sort_values('total_revenue')
ax2.barh(range(len(new_top10)), new_top10['total_revenue'], color='#3498db')
ax2.set_yticks(range(len(new_top10)))
ax2.set_yticklabels([f"{row['stockcode']}\n{row['description'][:30]}" 
                      for _, row in new_top10.iterrows()], fontsize=9)
ax2.set_xlabel('총 매출 (£)', fontsize=12)
ax2.set_title('New TOP 10 상품', fontsize=14, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
# plt.savefig('champions_vs_new_top_products.png', dpi=300, bbox_inches='tight')
plt.show()



# 4. 세그먼트별 TOP 5 상품 (주요 4개 세그먼트별 인기 상품 파악 -> 타겟 마케팅)
main_segments = ['Champions', 'Loyal', 'New', 'Risk']
fig, axes = plt.subplots(2, 2, figsize=(18, 14))
axes = axes.flatten()

for idx, segment in enumerate(main_segments):
    ax = axes[idx]
    segment_top5 = df_top_products[
        df_top_products['segment_name'] == segment
    ].nsmallest(5, 'revenue_rank').sort_values('total_revenue')
    
    ax.barh(range(len(segment_top5)), segment_top5['total_revenue'])
    ax.set_yticks(range(len(segment_top5)))
    ax.set_yticklabels([f"{row['description'][:40]}" 
                        for _, row in segment_top5.iterrows()], fontsize=10)
    ax.set_xlabel('총 매출 (£)', fontsize=11)
    ax.set_title(f'{segment} 세그먼트 TOP 5 상품', fontsize=13, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
# plt.savefig('main_segment_top5_products.png', dpi=300, bbox_inches='tight')
plt.show()




# 5. 세그먼트별 상위 20개 상품 구매 분포 히트맵 (한눈에 상품 선호도 패턴 확인)
top_products_overall = df_product_summary.groupby('stockcode').agg({
    'total_revenue': 'sum'
}).nlargest(20, 'total_revenue').index

# 세그먼트-상품 매트릭스 생성
heatmap_data = df_product_summary[
    df_product_summary['stockcode'].isin(top_products_overall)
].pivot_table(
    index='segment_name',
    columns='stockcode',
    values='total_revenue',
    fill_value=0
)

# 세그먼트 순서 정렬
heatmap_data = heatmap_data.reindex([s for s in segment_order if s in heatmap_data.index])

# 정규화 (세그먼트별 비율)
heatmap_normalized = heatmap_data.div(heatmap_data.sum(axis=1), axis=0) * 100

plt.figure(figsize=(16, 10))
sns.heatmap(heatmap_normalized, annot=True, fmt='.1f', cmap='YlOrRd', 
            cbar_kws={'label': '매출 비중 (%)'}, linewidths=0.5)
plt.title('세그먼트별 상위 20개 상품 구매 비중 히트맵 (%)', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('상품 코드', fontsize=12)
plt.ylabel('세그먼트', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
# plt.savefig('segment_product_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# 7. 인사이트 및 결과 저장(선택)
# =============================================================================

# 결과를 CSV로 저장
# df_purchase_patterns.to_csv('segment_purchase_patterns.csv', index=False, encoding='utf-8-sig')
# champions_new_summary.to_csv('champions_vs_new_summary.csv', index=False, encoding='utf-8-sig')

# Champions TOP 20 상품
# champions_top20 = df_champions_vs_new[
#     df_champions_vs_new['customer_group'] == 'Champions'
# ].nlargest(20, 'total_revenue')
# champions_top20.to_csv('champions_top20_products.csv', index=False, encoding='utf-8-sig')

# New TOP 20 상품
# new_top20 = df_champions_vs_new[
#     df_champions_vs_new['customer_group'] == 'New'
# ].nlargest(20, 'total_revenue')
# new_top20.to_csv('new_top20_products.csv', index=False, encoding='utf-8-sig')

# print("\n생성된 파일:")
# print("1. segment_purchase_patterns.png - 세그먼트별 구매 패턴")
# print("2. champions_vs_new_comparison.png - Champions vs New 비교")
# print("3. champions_vs_new_top_products.png - 그룹별 TOP 10 상품")
# print("4. segment_top5_products.png - 주요 세그먼트별 TOP 5 상품")
# print("5. segment_product_heatmap.png - 세그먼트-상품 히트맵")
# print("6. segment_purchase_patterns.csv - 세그먼트 구매 패턴 데이터")
# print("7. champions_vs_new_summary.csv - Champions vs New 요약 데이터")
# print("8. champions_top20_products.csv - Champions TOP 20 상품")
# print("9. new_top20_products.csv - New TOP 20 상품")